In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import random

from definitions import DATA_DIR
from concept_detection.text.io import ProgressBar

In [3]:
# Load successors adjacency list
print('Loading successors adjacency list...')
with open(f'{DATA_DIR}/successors.json') as f:
    successors = json.load(f)
successors = {int(k): v for k, v in successors.items()}
print('Loaded')

# Load predecessors adjacency list
print('Loading predecessors adjacency list...')
with open(f'{DATA_DIR}/predecessors.json') as f:
    predecessors = json.load(f)
predecessors = {int(k): v for k, v in predecessors.items()}
print('Loaded')

Loading successors adjacency list...
Loaded
Loading predecessors adjacency list...
Loaded


In [4]:
def get_random_pairs():
    sources = random.sample(successors.keys(), 100)
    targets = random.sample(predecessors.keys(), 100)

    return list(zip(sources, targets))

In [5]:
pairs = get_random_pairs()

In [6]:
def B(u, n):
    if n == 0:
        return {u}
    
    if n == 1:
        return {u} | set(successors[u])
    
    ball = set()
    for v in B(u, 1):
        ball |= B(v, n-1)

    return ball


def antiB(u, n):
    if n == 0:
        return {u}
    
    if n == 1:
        return {u} | set(predecessors[u])
    
    ball = set()
    for v in antiB(u, 1):
        ball |= antiB(v, n-1)

    return ball


def min_dist(s, t):    
    n = 0
    s_out_prev = set()
    t_in_prev = set()
    s_stable = False
    t_stable = False
    while True:
        if not s_stable:
            s_out = B(s, n)

            if s_out == s_out_prev:
                s_stable = True
            else:
                s_out_prev = s_out

        if not t_stable:
            t_in = antiB(t, n)

            if t_in == t_in_prev:
                t_stable = True
            else:
                t_in_prev = t_in
        
        if s_out & t_in:
            return n
        
        if s_stable or t_stable:
            return 999999999

        n += 1

In [133]:
min_dists = []

pb = ProgressBar(len(pairs))
for s, t in pairs:
    min_dists.append(min_dist(s, t))
    pb.update()

[##################################################] 100.00%


In [145]:
are_6_connected = []

pb = ProgressBar(len(pairs))
for s, t in pairs:
    are_6_connected.append(len(B(s, 3) & antiB(t, 3)) > 0)
    pb.update()

[##################################################] 100.00%


In [146]:
are_6_connected.count(True), are_6_connected.count(False)

(25, 75)

In [29]:
sources = []
sinks = []
isolated = []

for u in successors:
    if not successors[u]:
        sinks.append(u)
        
    if not predecessors[u]:
        sources.append(u)
    
    if not successors[u] and not predecessors[u]:
        isolated.append(u)

In [33]:
len(successors), len(predecessors), len(sources), len(sinks), len(isolated)

(5282817, 5282817, 6044, 3776202, 0)